![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_AlbertForZeroShotClassification.ipynb)

## Import AlbertForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 5.4.2` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for sequence classification via `AlbertForSequenceClassification` or `TFAlbertForSequenceClassification`. These models are usually under `Text Classification` category and have `albert` in their labels
- Reference: [TFAlbertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/albert#transformers.TFAlbertForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=albert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [4]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 45.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT'](https://huggingface.co/mohsenfayyaz/albert-base-v2-toxicity) model from HuggingFace as an example
- In addition to `TFAlbertForSequenceClassification` we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [5]:
from transformers import TFAlbertForSequenceClassification, AlbertTokenizer
import tensorflow as tf

MODEL_NAME = 'DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT'

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFAlbertForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFAlbertForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

try downloading PyTorch weights


pytorch_model.bin:   0%|          | 0.00/891M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForSequenceClassification: ['albert.embeddings.position_ids']
- This IS expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForSequenceClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {MODEL_NAME}

total 869888
-rw-r--r-- 1 root root      1787 Aug  7 18:39 config.json
drwxr-xr-x 3 root root      4096 Aug  7 18:39 saved_model
-rw-r--r-- 1 root root 890751296 Aug  7 18:39 tf_model.h5


In [7]:
!ls -l {MODEL_NAME}/saved_model/1

total 9500
drwxr-xr-x 2 root root    4096 Aug  7 18:39 assets
-rw-r--r-- 1 root root      53 Aug  7 18:39 fingerprint.pb
-rw-r--r-- 1 root root   26815 Aug  7 18:39 keras_metadata.pb
-rw-r--r-- 1 root root 9683747 Aug  7 18:39 saved_model.pb
drwxr-xr-x 2 root root    4096 Aug  7 18:39 variables


We are using based model for the tokenizer because the model `DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT` does not have sentencepiece

In [8]:
from transformers import AlbertTokenizer
import tensorflow as tf

# Try to load the tokenizer
try:
    tokenizer = AlbertTokenizer.from_pretrained('albert-xxlarge-v2')
    print("Tokenizer loaded successfully!")
except OSError as e:
    print(f"Error loading tokenizer: {e}")

# Try to save the tokenizer
try:
    tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))
    print("Tokenizer saved successfully!")
except Exception as e:
    print(f"Error saving tokenizer: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

Tokenizer loaded successfully!
Tokenizer saved successfully!


In [9]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    286 Aug  7 18:39 special_tokens_map.json
-rw-r--r-- 1 root root 760289 Aug  7 18:39 spiece.model
-rw-r--r-- 1 root root   1246 Aug  7 18:39 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [10]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {asset_path}

In [12]:
import os
import json

# Define human-readable labels (based on your example)
human_readable_labels = ["entailment", "contradict"]

# Get the auto-generated label mapping
labels = model.config.label2id

# Determine the maximum index in id2label
max_index = max(labels.values())

In [15]:
# Map auto-generated labels to human-readable labels
mapped_labels = []
for idx in range(max_index + 1):
    if idx < len(human_readable_labels):
        mapped_labels.append(human_readable_labels[idx])

In [16]:
mapped_labels

['entailment', 'contradict']

In [17]:
with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(mapped_labels))

In [18]:
!cat {asset_path}'/labels.txt'

entailment
contradict

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [19]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 748
-rw-r--r-- 1 root root     21 Aug  7 18:39 labels.txt
-rw-r--r-- 1 root root 760289 Aug  7 18:39 spiece.model


In [20]:
!ls -lR {MODEL_NAME}/saved_model/1

DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT/saved_model/1:
total 9500
drwxr-xr-x 2 root root    4096 Aug  7 18:39 assets
-rw-r--r-- 1 root root      53 Aug  7 18:39 fingerprint.pb
-rw-r--r-- 1 root root   26815 Aug  7 18:39 keras_metadata.pb
-rw-r--r-- 1 root root 9683747 Aug  7 18:39 saved_model.pb
drwxr-xr-x 2 root root    4096 Aug  7 18:39 variables

DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT/saved_model/1/assets:
total 748
-rw-r--r-- 1 root root     21 Aug  7 18:39 labels.txt
-rw-r--r-- 1 root root 760289 Aug  7 18:39 spiece.model

DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT/saved_model/1/variables:
total 869864
-rw-r--r-- 1 root root 890729740 Aug  7 18:39 variables.data-00000-of-00001
-rw-r--r-- 1 root root      1711 Aug  7 18:39 variables.index


## Import and Save AlbertForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [22]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [27]:
# import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


Apache Spark version: 3.4.0


- Let's use `loadSavedModel` functon in `AlbertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [29]:
from sparknlp.annotator import *

zero_shot_classifier = AlbertForZeroShotClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)\
  .setCandidateLabels(["urgent", "mobile", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [30]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [31]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your AlbertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [32]:
! ls -l {MODEL_NAME}_spark_nlp

total 880148
-rw-r--r-- 1 root root 900495299 Aug  7 18:44 albert_classification_tensorflow
-rw-r--r-- 1 root root    760289 Aug  7 18:44 albert_spp
drwxr-xr-x 4 root root      4096 Aug  7 18:44 fields
drwxr-xr-x 2 root root      4096 Aug  7 18:44 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForZeroShotClassification model 😊

In [33]:
sequenceClassifier_loaded = AlbertForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("multi_class") \
  .setCandidateLabels(["urgent", "mobile", "technology"])

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [35]:
from pyspark.ml import Pipeline

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["I have a problem with my iphone that needs to be resolved asap!!"]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "multi_class.result").show()

+--------------------+--------+
|                text|  result|
+--------------------+--------+
|I have a problem ...|[urgent]|
+--------------------+--------+



That's it! You can now go wild and use hundreds of `AlbertForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
